In this notebook we build a quantitative value investment strategy.

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

More precisely, we will

- select the 50 stocks (among the stocks composing S \& P 500) with the highest "value"

- where the "value" will be a "robust value score" computed based on various measures and ratios and

- compute recommended trades to build an "equal-weight" position on these 50 stocks.

We start by retrieving (from wikipedia) the composition of the S \& P 500 and store it in a list

In [1]:
import pandas as pd

def get_sp_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    sp500_tickers = sp500_table["Symbol"].tolist()
    sp500_tickers = [ticker.replace('.', '-') for ticker in sp500_tickers] # reformat the strings (e.g. BF.B -> BF-B) for yfinance
    return sp500_tickers

sp500_tickers = get_sp_tickers()
sp500_tickers[:5]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']

We now use yahoo finance api to get the ratios of interest of the S \& P 500 stocks, namely

- price-to-earnings ratio (p/e)
- price-to-book ratio (p/b)
- price-to-sales ratio (p/s)
- enterprise value-to-earnings before interest, taxes, depreciation and amortization ratio (ev/ebitda)
- enterprise value-to-gross profits ratio (ev/gp)

The Tickers class of yfinance do not support batch api calls (that is, can only feed him one ticker string), hence we need to loop through all S \& P 500 tickers. However we can speed up the process using multi-threading

In [2]:
import yfinance as yf
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def get_ratios(tickers):
    def get_stats(ticker):
        try:
            info = yf.Tickers(ticker).tickers[ticker].info
            price = info.get("currentPrice")
            pe = info.get("trailingPE")
            pb = info.get("priceToBook")
            ps = info.get("priceToSalesTrailing12Months")
            ev_to_ebitda = info.get("enterpriseToEbitda")
            ev = info.get("enterpriseValue")
            gp = info.get("grossProfits")
            ev_gp = ev / gp if ev and gp else None

            return {
                "Ticker": ticker,
                "Price": price,
                "Strategy (nb shares to buy)": "N/A",
                "P/E ratio": pe,
                "P/E percentile": "N/A",
                "P/B ratio": pb,
                "P/B percentile": "N/A",
                "P/S ratio": ps,
                "P/S percentile": "N/A",
                "EV/EBITDA ratio": ev_to_ebitda,
                "EV/EBITDA percentile": "N/A",
                "EV/GP ratio": ev_gp,
                "EV/GP percentile": "N/A",
                "RV score": "N/A"
            }
        except Exception as e:
            print(f"Error retrieving data for {ticker}: {e}")
            return None

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(get_stats, tickers))

    filtered_results = [r for r in results if r is not None] # Filter out any None results due to errors

    out = pd.DataFrame(filtered_results)
    return out

sp_ratios = get_ratios(sp500_tickers)
sp_ratios.head()

,Ticker,Price,Strategy (nb shares to buy),P/E ratio,P/E percentile,P/B ratio,P/B percentile,P/S ratio,P/S percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/GP ratio,EV/GP percentile,RV score
0,MMM,135.95,N/A,18.725895,N/A,19.088737,N/A,2.983531,N/A,14.587,N/A,7.854917,N/A,N/A
1,AOS,64.50,N/A,17.768595,N/A,4.942150,N/A,2.424667,N/A,11.881,N/A,6.344184,N/A,N/A
2,ABT,126.88,N/A,16.607330,N/A,4.609795,N/A,5.245543,N/A,21.243,N/A,9.765509,N/A,N/A
3,ABBV,175.05,N/A,72.937500,N/A,92.914020,N/A,5.496857,N/A,13.906,N/A,9.371380,N/A,N/A
4,ACN,284.34,N/A,23.421745,N/A,6.090477,N/A,2.648026,N/A,15.559,N/A,8.251233,N/A,N/A


In [3]:
sp_ratios.isna().sum()

Ticker                          0
Price                           0
Strategy (nb shares to buy)     0
P/E ratio                      22
P/E percentile                  0
P/B ratio                       0
P/B percentile                  0
P/S ratio                       0
P/S percentile                  0
EV/EBITDA ratio                32
EV/EBITDA percentile            0
EV/GP ratio                     4
EV/GP percentile                0
RV score                        0
dtype: int64

We handle the few nan value by filling them by the mean of the column

In [4]:
for c in [c for c in sp_ratios.columns if sp_ratios[c].dtype == "float64"]:
    sp_ratios[c] = sp_ratios[c].fillna(sp_ratios[c].mean())

sp_ratios.head()

,Ticker,Price,Strategy (nb shares to buy),P/E ratio,P/E percentile,P/B ratio,P/B percentile,P/S ratio,P/S percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/GP ratio,EV/GP percentile,RV score
0,MMM,135.95,N/A,18.725895,N/A,19.088737,N/A,2.983531,N/A,14.587,N/A,7.854917,N/A,N/A
1,AOS,64.50,N/A,17.768595,N/A,4.942150,N/A,2.424667,N/A,11.881,N/A,6.344184,N/A,N/A
2,ABT,126.88,N/A,16.607330,N/A,4.609795,N/A,5.245543,N/A,21.243,N/A,9.765509,N/A,N/A
3,ABBV,175.05,N/A,72.937500,N/A,92.914020,N/A,5.496857,N/A,13.906,N/A,9.371380,N/A,N/A
4,ACN,284.34,N/A,23.421745,N/A,6.090477,N/A,2.648026,N/A,15.559,N/A,8.251233,N/A,N/A


In [5]:
sp_ratios.isna().sum()

Ticker                         0
Price                          0
Strategy (nb shares to buy)    0
P/E ratio                      0
P/E percentile                 0
P/B ratio                      0
P/B percentile                 0
P/S ratio                      0
P/S percentile                 0
EV/EBITDA ratio                0
EV/EBITDA percentile           0
EV/GP ratio                    0
EV/GP percentile               0
RV score                       0
dtype: int64

We now compute value percentiles using scipy.stats's percentileofscore method. For example, if Apple stock has 80 "P/S percentile", this  means that 80% of the "P/S ratio" are below Apple's "P/S ratio".

In [6]:
from scipy import stats

ratios = ["P/E", "P/B", "P/S", "EV/EBITDA", "EV/GP"]
for row in sp_ratios.index:
    for ratio in ratios:
        ratio_col = f"{ratio} ratio"
        percentile_col = f"{ratio} percentile"
        sp_ratios.loc[row, percentile_col] = stats.percentileofscore(sp_ratios[ratio_col], sp_ratios.loc[row, ratio_col])
sp_ratios.head()

,Ticker,Price,Strategy (nb shares to buy),P/E ratio,P/E percentile,P/B ratio,P/B percentile,P/S ratio,P/S percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/GP ratio,EV/GP percentile,RV score
0,MMM,135.95,N/A,18.725895,35.586481,19.088737,93.836978,2.983531,51.491054,14.587,47.912525,7.854917,49.900596,N/A
1,AOS,64.50,N/A,17.768595,32.405567,4.942150,66.40159,2.424667,42.345924,11.881,34.194831,6.344184,33.996024,N/A
2,ABT,126.88,N/A,16.607330,28.031809,4.609795,64.413519,5.245543,73.558648,21.243,82.107356,9.765509,60.636183,N/A
3,ABBV,175.05,N/A,72.937500,94.4334,92.914020,99.005964,5.496857,75.347913,13.906,44.135189,9.371380,58.05169,N/A
4,ACN,284.34,N/A,23.421745,48.906561,6.090477,73.161034,2.648026,45.526839,15.559,51.491054,8.251233,52.286282,N/A


Now, we compute the "robust value score" of the stocks: we choose the arithmetic mean of our ratios

In [7]:

for row in sp_ratios.index:
    rv_score = 0
    for ratio in ratios:
        percentile_col = f"{ratio} percentile"
        rv_score += sp_ratios.loc[row, f"{ratio} percentile"]
    sp_ratios.loc[row, 'RV score'] = rv_score / len(ratios)
    
sp_ratios.head()

,Ticker,Price,Strategy (nb shares to buy),P/E ratio,P/E percentile,P/B ratio,P/B percentile,P/S ratio,P/S percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/GP ratio,EV/GP percentile,RV score
0,MMM,135.95,N/A,18.725895,35.586481,19.088737,93.836978,2.983531,51.491054,14.587,47.912525,7.854917,49.900596,55.745527
1,AOS,64.50,N/A,17.768595,32.405567,4.942150,66.40159,2.424667,42.345924,11.881,34.194831,6.344184,33.996024,41.868787
2,ABT,126.88,N/A,16.607330,28.031809,4.609795,64.413519,5.245543,73.558648,21.243,82.107356,9.765509,60.636183,61.749503
3,ABBV,175.05,N/A,72.937500,94.4334,92.914020,99.005964,5.496857,75.347913,13.906,44.135189,9.371380,58.05169,74.194831
4,ACN,284.34,N/A,23.421745,48.906561,6.090477,73.161034,2.648026,45.526839,15.559,51.491054,8.251233,52.286282,54.274354


and finally select the top 50 "RV score"

In [8]:
sp_ratios.sort_values(by = 'RV score', ascending = False, inplace = True)
sp_ratios = sp_ratios[:50]
sp_ratios.reset_index(drop = True, inplace = True)

sp_ratios.head()

,Ticker,Price,Strategy (nb shares to buy),P/E ratio,P/E percentile,P/B ratio,P/B percentile,P/S ratio,P/S percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/GP ratio,EV/GP percentile,RV score
0,PLTR,88.55,N/A,466.052640,99.005964,41.397854,97.415507,72.476395,100.0,592.946,99.801193,88.184473,99.801193,99.204771
1,AXON,567.98,N/A,117.594200,97.415507,18.695852,93.638171,21.181095,99.204771,345.365,99.602386,34.835460,99.204771,97.813121
2,CRWD,377.90,N/A,65.084342,91.153082,28.561710,96.620278,23.692492,99.403579,888.210,100.0,30.442956,98.210736,97.077535
3,TPL,1236.10,N/A,62.492413,88.866799,25.074040,96.222664,40.253025,99.801193,49.242,97.017893,42.508665,99.403579,96.262425
4,NOW,785.67,N/A,114.864030,97.017893,16.883059,92.047714,14.806560,97.415507,80.757,98.807157,18.209041,92.445328,95.54672


We can now build an equally weighted portfolio of these 50 stocks, for a given portfolio size

In [9]:
import math

porfolio_size = 1000000
position_size = 1000000 / len(sp_ratios)

strategy = sp_ratios.copy()
for row in sp_ratios.index:
    strategy.loc[row, "Strategy (nb shares to buy)"] = math.floor(position_size / sp_ratios.loc[row, "Price"])
strategy.head()

,Ticker,Price,Strategy (nb shares to buy),P/E ratio,P/E percentile,P/B ratio,P/B percentile,P/S ratio,P/S percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/GP ratio,EV/GP percentile,RV score
0,PLTR,88.55,225,466.052640,99.005964,41.397854,97.415507,72.476395,100.0,592.946,99.801193,88.184473,99.801193,99.204771
1,AXON,567.98,35,117.594200,97.415507,18.695852,93.638171,21.181095,99.204771,345.365,99.602386,34.835460,99.204771,97.813121
2,CRWD,377.90,52,65.084342,91.153082,28.561710,96.620278,23.692492,99.403579,888.210,100.0,30.442956,98.210736,97.077535
3,TPL,1236.10,16,62.492413,88.866799,25.074040,96.222664,40.253025,99.801193,49.242,97.017893,42.508665,99.403579,96.262425
4,NOW,785.67,25,114.864030,97.017893,16.883059,92.047714,14.806560,97.415507,80.757,98.807157,18.209041,92.445328,95.54672
